In [1]:
import pandas as pd
import os
import numpy as np
import pandas as pd
import sklearn
import numpy as np
import lightgbm
from sklearn.model_selection import train_test_split
import joblib
import scipy as sp
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from sklearn.linear_model import LogisticRegression, Ridge, RidgeClassifier

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt


In [2]:
from catboost import CatBoostClassifier, Pool, cv
from sklearn.metrics import accuracy_score

In [3]:
train = pd.read_csv("../data/onti_p.csv")
train.head()

,Coolness_RHEED,R FWHM_RHEED,X FWHM_RHEED,Y FWHM_RHEED,Crystal Position,Displayed Rate,Filtered Rate,Length,Raw Rate,Source Power,Time
0,0.0,0.0,0.0,0.0,1.0,6.027,6.000,0.003721,5.990,21.64,2021-02-23 02:21:05
1,0.0,0.0,0.0,0.0,1.0,6.070,6.094,0.028870,6.110,21.06,2021-02-23 02:21:06
2,0.0,0.0,0.0,0.0,1.0,6.156,6.082,0.040400,6.168,20.97,2021-02-23 02:21:07
3,0.0,0.0,0.0,0.0,1.0,6.110,6.168,0.067750,6.156,20.70,2021-02-23 02:21:08
4,0.0,0.0,0.0,0.0,1.0,6.094,6.168,0.083560,6.145,20.60,2021-02-23 02:21:09


In [4]:
#важные колонки для использования
val_col = ['Coolness_RHEED',  'R FWHM_RHEED',  'X FWHM_RHEED', 'Y FWHM_RHEED',
        'Filtered Rate', 'Displayed Rate', 'Raw Rate',
       'Source Power', 'Crystal Position']


In [5]:
#создаем датасет разными аггрегационными функциями
def generated_dataset(series, wind_size=5):
    result = series.rolling(wind_size).agg(['mean', 'std', 'var'])
    delta = series - series.shift(wind_size)
    delta.columns = [col+'_'+str(wind_size) for col in delta.columns]
    result.columns = [col[0]+'_'+col[1]+'_'+str(wind_size) for col in result.columns]
    result = result.join(delta)
    return result

In [6]:
df_X = generated_dataset(train[val_col])
df_X.shape, df_X.columns

((2363772, 36),
 Index(['Coolness_RHEED_mean_5', 'Coolness_RHEED_std_5', 'Coolness_RHEED_var_5',
        'R FWHM_RHEED_mean_5', 'R FWHM_RHEED_std_5', 'R FWHM_RHEED_var_5',
        'X FWHM_RHEED_mean_5', 'X FWHM_RHEED_std_5', 'X FWHM_RHEED_var_5',
        'Y FWHM_RHEED_mean_5', 'Y FWHM_RHEED_std_5', 'Y FWHM_RHEED_var_5',
        'Filtered Rate_mean_5', 'Filtered Rate_std_5', 'Filtered Rate_var_5',
        'Displayed Rate_mean_5', 'Displayed Rate_std_5', 'Displayed Rate_var_5',
        'Raw Rate_mean_5', 'Raw Rate_std_5', 'Raw Rate_var_5',
        'Source Power_mean_5', 'Source Power_std_5', 'Source Power_var_5',
        'Crystal Position_mean_5', 'Crystal Position_std_5',
        'Crystal Position_var_5', 'Coolness_RHEED_5', 'R FWHM_RHEED_5',
        'X FWHM_RHEED_5', 'Y FWHM_RHEED_5', 'Filtered Rate_5',
        'Displayed Rate_5', 'Raw Rate_5', 'Source Power_5',
        'Crystal Position_5'],
       dtype='object'))

In [7]:
df_X = df_X.join(generated_dataset(train[val_col],wind_size=50))
df_X.shape

(2363772, 72)

In [8]:
df_X = df_X.join(generated_dataset(train[val_col],wind_size=500))
df_X.shape

(2363772, 108)

In [9]:
df_X = df_X.join(generated_dataset(train[val_col],wind_size=1500))
df_X.shape

(2363772, 144)

In [10]:
mask_base = df_X.isna().sum(axis=1)==0
df_X.shape

(2363772, 144)

In [11]:
mask_base.sum()

2287456

In [ ]:
wind_size = 435

df = df_X[wind_size:-wind_size]
#y = (rheed_long.Coolness < rheed_long.Coolness.shift(wind_size) )
#y = y[wind_size:-wind_size]
cool_col = 'Coolness_RHEED_mean_50'
y = (abs((df[cool_col].shift(wind_size) -  df[cool_col]))>5)


mask = mask_base #&(y>30)
df = df[mask]
y = y[mask]
categorical_features_indices = np.where(df.dtypes != np.float)[0]
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.10, shuffle=False)

In [ ]:
import hyperopt

def hyperopt_objective(params):
    model = CatBoostClassifier(
        l2_leaf_reg=int(params['l2_leaf_reg']),
        learning_rate=params['learning_rate'],
        iterations=10,
        eval_metric='AUC',
        random_seed=42,
        verbose=False,
        loss_function='Logloss',
    )
    
    cv_data = cv(
        Pool(X_train, y_train, cat_features=categorical_features_indices),
        model.get_params()
    )
    best_accuracy = np.max(cv_data['test-AUC-mean'])
    
    return 1 - best_accuracy # as hyperopt minimises

In [ ]:
from numpy.random import RandomState

params_space = {
    'l2_leaf_reg': hyperopt.hp.qloguniform('l2_leaf_reg', 0, 2, 1),
    'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-3, 5e-1),
    'colsample_bylevel': hyperopt.hp.uniform('colsample_bylevel', 0.5, 1.0),
    'bagging_temperature': hyperopt.hp.uniform('bagging_temperature', 0.0, 100),
    'random_strength': hyperopt.hp.uniform('random_strength', 0.0, 100),
    'n_estimators': hyperopt.hp.choice('n_estimators', np.arange(50, 250, 25)),
    'max_depth': hyperopt.hp.choice('max_depth', np.arange(5, 11)),
    'reg_lambda': hyperopt.hp.uniform('reg_lambda', 1, 10),
    'scale_pos_weight': hyperopt.hp.uniform('scale_pos_weight', 1, 50)
}

trials = hyperopt.Trials()

best = hyperopt.fmin(
    hyperopt_objective,
    space=params_space,
    algo=hyperopt.tpe.suggest,
    max_evals=1500,
    trials=trials,
    rstate=RandomState(42)
)

print(best)

In [ ]:
model = CatBoostClassifier(
#     l2_leaf_reg=int(best['l2_leaf_reg']),
#     learning_rate=best['learning_rate'],
    iterations=15000,
    eval_metric='AUC',
    random_seed=42,
    verbose=True,
    loss_function='Logloss',
    od_type='Iter',
    od_wait=150
)

train_pool = Pool(X_train, y_train, cat_features=categorical_features_indices)
validate_pool = Pool(X_test, y_test, cat_features=categorical_features_indices)

model.fit(train_pool, eval_set=validate_pool)

In [ ]:
import pickle
with open("../saves/catboost3.pkl","wb") as f:
    pickle.dump(model, f)

In [ ]:
fpr, tpr, _ = sklearn.metrics.roc_curve(y_test, model.predict_proba(X_test)[:,1])
roc_auc = sklearn.metrics.roc_auc_score(y_test, model.predict_proba(X_test)[:,1])
plt.figure()
lw = 2
plt.plot(fpr, tpr,color='darkorange', lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [ ]:
roc_auc